In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove, chdir
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent))
current_dir = getcwd()
print(current_dir)

In [ ]:
import pandas as pd
from random import randint
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)

In [ ]:
from bulletin import root, default_input, default_output, agora, hoje, ontem, anteontem, dias_apos, dias_apos_label
from bulletin.services.metabase import Metabase
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils import utils, static
from bulletin.utils.xls_ import fit_cols
from bulletin.utils.normalize import trim_overspace
from bulletin.utils.normalize import normalize_text#normalize_hash, normalize_labels, , date_hash, normalize_number

In [ ]:
from datetime import datetime, date, timedelta

exclusao_pathfile = join(root, 'database', 'casos_confirmados')

today = pd.to_datetime(date.today())
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)
data_retroativos = ontem - timedelta(31)

In [ ]:

utils.create_backup(first_name = "backup_notifica_diario_" , level=3)
chdir(current_dir)

In [ ]:
from bulletin.utils.clean_up import clear_directories
clear_directories()

In [ ]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio']
municipios.loc[municipios['uf']!='PR','mun_resid'] = municipios.loc[municipios['uf']!='PR','municipio'] + '/' + municipios['uf']

In [ ]:
update = False
load_downloaded = False

# Load 
notifica = Notifica()
notifica.databases()
notifica.load('notifica', compress=False)
notifica.df = notifica.df.drop_duplicates('id', keep='last')
    
if update:
    mb = Metabase()
    days = 21
    intervalo = f"(data_notificacao >= NOW() - INTERVAL '{days} DAY') or (data_liberacao >= NOW() - INTERVAL '{days} DAY') or (updated_at >= NOW() - INTERVAL '{days} DAY') or (data_coleta >= NOW() - INTERVAL '{days} DAY') or (data_encerramento >= NOW() - INTERVAL '{days} DAY') or (data_cura_obito >= NOW() - INTERVAL '{days} DAY')"
    mb.generate_notifica_query('update_notifica', where=intervalo, replace=True)
    update_notifica_parts = mb.download_notificacao('update_notifica', load=load_downloaded)
    
    update_notifica = Notifica()
    update_notifica.read(update_notifica_parts)
    update_notifica.normalize()
    
    
    update_notifica.df = update_notifica.df.drop_duplicates('id', keep='last')
    analise_dtd = update_notifica.analise_data_diagnostico() ## only in casos confirmados, pd.NaT nos demais

#     novas_notificacoes, atualizacoes_fichas = notifica.update(update_notifica)#, observed_cols=['excluir_ficha','status_notificacao','classificacao_final','data_1o_sintomas','data_diagnostico','evolucao','data_cura_obito','data_encerramento'])
    notifica.update(update_notifica)

    notifica.fix_dtypes()
    notifica.save(replace=True, compress=False)

print(len(notifica.df))

In [ ]:
# # notifica.df = notifica.df.loc[((notifica.df['classificacao_final']==2)&(notifica.df['excluir_ficha']==2)&(notifica.df['status_notificacao'].isin([1,2])))]

# notifica.df = pd.merge(notifica.df.rename(columns={'ibge_residencia':'ibge'}),municipios[['ibge','macro','rs','mun_resid','uf','municipio','regional']],on='ibge',how='left').rename(columns={'ibge':'ibge_residencia'})
# notifica.df = pd.merge(notifica.df.rename(columns={'ibge_unidade_notifica':'ibge'}),municipios[['ibge','mun_resid']].rename(columns={'mun_resid':'mun_atend'}),on='ibge',how='left').rename(columns={'ibge':'ibge_unidade_notifica'})

# # notifica.df['exame_nome'] = notifica.toggle_merge_table('exame',inplace=False)
# notifica.toggle_merge_table('sexo')

# notifica.df = notifica.df.loc[((notifica.df['sexo']!='N')&(notifica.df['mun_resid'].notna())&(notifica.df['data_diagnostico'].notna())&(notifica.df['paciente'].str.len() > 5))]
# # notifica.df

In [ ]:
cc = CasosConfirmados()
cc.load(f"cc_{hoje.strftime('%d_%m_%Y')}", compress=True)
cc.df['excluir_ficha'] = 2 #EXCLUIR FICHA "NÃO"
cc.df['status_notificacao'] = 2 #STATUS NOTIFICAÇÃO "ENCERRADA"

# df_exclusao_old = pd.DataFrame()

In [ ]:
casos_confirmados = cc.df.loc[cc.df['id_notifica'] > 0].copy()
casos_confirmados['classificacao_final'] = 2

obitos_confirmados = casos_confirmados.loc[casos_confirmados['evolucao']==2]
nao_obitos = casos_confirmados.loc[casos_confirmados['evolucao']!=2]

In [ ]:
## CASOS (NÃO ÓBITOS) QUE EVOLUÍRAM PARA ÓBITO NO NOTIFICA

In [ ]:
diff_cc_final = pd.DataFrame()
diff_cc_final = pd.merge(nao_obitos[['id_notifica','paciente','evolucao','status_notificacao']], notifica.df[['id', 'evolucao', 'status_notificacao']].rename(columns={'id':'id_notifica'}), on='id_notifica', how='inner', suffixes=['_old','_new'])

diff_cc_final = diff_cc_final.loc[diff_cc_final['evolucao_old'] != diff_cc_final['evolucao_new']]
diff_cc_final

In [ ]:
obitos_inserir = diff_cc_final.loc[ (diff_cc_final['evolucao_old'].isin([1,3])) & (diff_cc_final['evolucao_new'] == 2) ]
obitos_inserir

In [ ]:
diff_cc_final.loc[ (diff_cc_final['evolucao_old'] == 3) & (diff_cc_final['evolucao_new'] == 1) ]

In [ ]:
obitos_inserir['id_notifica'].to_list()

In [ ]:
len(obitos_inserir.loc[obitos_inserir['status_notificacao_new'] == 2])

In [ ]:
path = join('C:\\', 'SESA', 'Inconsistencias')

print(len(obitos_inserir))
obitos_inserir.to_excel(join(path, 'obitos_inserir.xlsx'), index=False)